In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
import yaml
import json
import time
from datetime import datetime
import multiprocessing as mp
import statsmodels.api as sm
from functools import partial
# import dolphindb as ddb
from itertools import chain, product 
from threadpoolctl import threadpool_limits 
from sklearn.linear_model import LinearRegression
import asyncpg
# from PNL import *
from flib.utils import read_supplementary_data, read_target_return, transform_column_type
from flib import DailyReader
from utils import *
import flib
import random
# pd.set_option('display.max_rows', None)  # 显示所有行
# from PNL import PNL_STOCK
warnings.filterwarnings("ignore")
# from pilot import *
from tqdm import tqdm

In [2]:
path_lst = os.listdir('/home/yicheng/credit/data')

In [3]:
df_all = pd.DataFrame()
for path  in tqdm(path_lst):
    a = pd.read_parquet(f'/home/yicheng/credit/data/{path}')
    use_lst = ['ticker','date','tenor','parspread','upfront']
    b = a.loc[a['tenor']=='5Y']
    temp = b[use_lst]
    # temp['path'] = path
    df_all = pd.concat([df_all,temp])



100%|██████████| 5/5 [00:00<00:00, 12.47it/s]


In [ ]:
df_all.sort_values(by=['date',],inplace=False).groupby('date')

,ticker,tenor,parspread,upfront
date,,,,
2021-01-01,XRX,5Y,0.017344,-0.154914
2021-01-04,XRX,5Y,0.017354,-0.154583
2021-01-05,XRX,5Y,0.017326,-0.154758
2021-01-06,XRX,5Y,0.017109,0.033659
2021-01-07,XRX,5Y,0.016987,0.033055
...,...,...,...,...
2025-09-08,XRX,5Y,0.212217,0.397754
2025-09-09,XRX,5Y,0.218331,0.407366
2025-09-10,XRX,5Y,0.224795,0.416382


In [ ]:
a.columns.tolist()

['ticker',
 'date',
 'batch',
 'shortname',
 'redcode',
 'sector',
 'region',
 'country',
 'avrating',
 'impliedrating',
 'tier',
 'currency',
 'docclause',
 'primarycurve',
 'primarypricetype',
 'runningcoupon',
 'primarycoupon',
 'tenor',
 'parspread',
 'convspreard',
 'upfront',
 'cdsrealrecovery',
 'cdsassumedrecovery',
 'carriedforward',
 'compositedepth5y',
 'primarydepthcontributed',
 'primarydepthpassed',
 'primarydepthpassed1wma',
 'primarydepthpassed1mma',
 'primarydepthpassed2mma',
 'compositepricerating',
 'compositecurverating',
 'hasclearsubmissions',
 'hasquotes',
 'hasbankcontributions',
 'clearingsubmissionsdepthcontr',
 'quotesdepthcontr',
 'bankcontributionsdepthcontr',
 'clearingsubmissionsdepthpassed',
 'quotesdepthpassed',
 'bankcontributionsdepthpassed',
 'dailyupdatefrequency',
 'maxdailymovement',
 '_52wkhigh5y',
 '_52wklow5y',
 'contrstddevpassedrecovery',
 'contrlqrangepassed',
 'contrlqrangepassedrecovery',
 'contr10thpctlpassed',
 'contr10pctlpassedrecovery

In [6]:
a = pd.read_parquet('/home/yicheng/credit/data/mkidwsfpi2qmdzop.parquet')
use_lst = ['ticker','date','tenor','parspread','upfront']
b = a.loc[a['tenor']=='5Y']

In [32]:
set(use_lst) - set(b.columns)

set()

In [33]:
b[use_lst]

,ticker,date,tenor,parspread,upfront
162,XRX,2021-01-01,5Y,0.017650,0.036259
163,XRX,2021-01-01,5Y,0.017650,-0.153332
164,XRX,2021-01-01,5Y,0.017812,0.037005
165,XRX,2021-01-01,5Y,0.017812,-0.152473
166,XRX,2021-01-01,5Y,0.017812,0.037005
...,...,...,...,...,...
63241,XRX,2021-12-31,5Y,0.020569,-0.135467
63242,XRX,2021-12-31,5Y,0.020569,0.048645
63243,XRX,2021-12-31,5Y,0.020569,-0.135467
63244,XRX,2021-12-31,5Y,0.020569,0.048645


In [3]:
reader = DailyReader('option_factors_henry')

In [4]:
reader.field_idx

['stock_tot_volume',
 'stock_tot_amount',
 'so_vol_ratio',
 'log_so_vol_ratio',
 'option_tot_volume',
 'pcratio',
 'tot_open_interest',
 'iv_slope',
 'skewiv',
 'minute_rv',
 'daily_rv',
 'C_1_A_iv',
 'C_1_B_iv',
 'C_1_C_iv',
 'C_1_D_iv',
 'C_1_E_iv',
 'C_1_F_iv',
 'C_1_G_iv',
 'C_1_H_iv',
 'C_1_I_iv',
 'C_1_J_iv',
 'C_1_K_iv',
 'C_1_L_iv',
 'C_1_M_iv',
 'C_1_N_iv',
 'C_1_O_iv',
 'C_1_P_iv',
 'C_1_Q_iv',
 'C_2_A_iv',
 'C_2_B_iv',
 'C_2_C_iv',
 'C_2_D_iv',
 'C_2_E_iv',
 'C_2_F_iv',
 'C_2_G_iv',
 'C_2_H_iv',
 'C_2_I_iv',
 'C_2_J_iv',
 'C_2_K_iv',
 'C_2_L_iv',
 'C_2_M_iv',
 'C_2_N_iv',
 'C_2_O_iv',
 'C_2_P_iv',
 'C_2_Q_iv',
 'C_3_A_iv',
 'C_3_B_iv',
 'C_3_C_iv',
 'C_3_D_iv',
 'C_3_E_iv',
 'C_3_F_iv',
 'C_3_G_iv',
 'C_3_H_iv',
 'C_3_I_iv',
 'C_3_J_iv',
 'C_3_K_iv',
 'C_3_L_iv',
 'C_3_M_iv',
 'C_3_N_iv',
 'C_3_O_iv',
 'C_3_P_iv',
 'C_3_Q_iv',
 'P_1_A_iv',
 'P_1_B_iv',
 'P_1_C_iv',
 'P_1_D_iv',
 'P_1_E_iv',
 'P_1_F_iv',
 'P_1_G_iv',
 'P_1_H_iv',
 'P_1_I_iv',
 'P_1_J_iv',
 'P_1_K_iv',
 'P_1_L